# Optymalizacja modelu przy pomocy macierzy kosztów

Zbiór do analizy https://www.kaggle.com/datasets/whenamancodes/fraud-detection

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import optuna

In [2]:
# puść ten kod, 
# jeżeli wywołujesz plik  w folderze rozwiąznaia, 
# a ramka danych znajduje się w folderze data
import os 
os.chdir('../')

In [3]:
# Wczytanie danych
df = pd.read_csv('data/credit_fraud_sample.csv')
del df['Unnamed: 0']

In [ ]:
# nagłówek
df.head()

In [ ]:
# korelacja
df.corr(method='spearman')['Class']

In [ ]:
# describe
df.describe()

In [8]:
# Zmienne w modelu
x_names = ['V1','V2','V3','V4','V5','V6','V7']

In [9]:
# Podział na zbiór treningowy i testowy
train_x, test_x,train_y, test_y = train_test_split(df[x_names],df['Class'], test_size=0.3, random_state=1000)

## Wagi w modelu

In [10]:
m1 = LogisticRegression(class_weight={0:1,1:10}).fit(train_x,train_y)

In [ ]:
m1.score(test_x,test_y)

## Macierz kosztów

1. Błędna prognoza frauda, który w rzeczywistości jest typową transakcją oznacza niepotrzebny koszt pracownika w wysokości 50. (FP)
2. Błędna prognoza, że transakcja nie jest fraudem kosztuje instytucje 1000, z uwagi na koszty prawne. (FN)

In [12]:
# Definicje kosztów
fp_cost = 50
fn_cost = 1000

In [16]:
def objective(trial: optuna.Trial):

    params = {'n_estimators': trial.suggest_int('n_estimators',10,200),
              'min_samples_split': trial.suggest_int('min_samples_split',5,100),
              'max_leaf_nodes': trial.suggest_int('max_leaf_nodes',20,100),
              'min_impurity_decrease': trial.suggest_float('min_impurity_decrease',0,1)}

    model = RandomForestClassifier(**params, random_state=123).fit(train_x,train_y)
    df_preds = pd.DataFrame(data=test_y.reset_index(drop=True))
    df_preds['pred'] = model.predict(test_x)
    fp = ((df_preds['pred']==1) & df_preds['Class']==0).astype(int).sum() * fp_cost
    fn = ((df_preds['pred']==0) & df_preds['Class']==1).astype(int).sum() * fn_cost
    return fp+fn

In [ ]:
# Stworzenie optymalizacji
study = optuna.create_study(direction='minimize',study_name='Cost_matrix_opt', sampler = optuna.samplers.NSGAIISampler())
study.optimize(objective, n_trials=50)

In [19]:
# Stworzenie modelu
model = RandomForestClassifier(**study.best_params, random_state=123).fit(train_x,train_y)

In [20]:
# Predykcje 
df_preds = pd.DataFrame(data=test_y.reset_index(drop=True))
df_preds['pred'] = model.predict(test_x)

In [21]:
# Sprawdzenie wyniku fukncji
fp = ((df_preds['pred']==1) & df_preds['Class']==0).astype(int).sum() * fp_cost
fn = ((df_preds['pred']==0) & df_preds['Class']==1).astype(int).sum() * fn_cost

In [ ]:
fp

In [ ]:
fn

In [ ]:
fp+fn

In [ ]:
# Accuracy score
accuracy_score(df_preds['Class'],df_preds['pred'])